<a href="https://colab.research.google.com/github/kravipa1/cakechat/blob/master/kohya_ss_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Train with Kohya's Stable Diffusion Trainers
%cd /content

from google.colab import drive
drive.mount('/content/drive')

!pip install dadaptation==3.1 diffusers[torch]==0.17.1 easygui==0.98.3 einops==0.6.0 fairscale==0.4.13 ftfy==6.1.1 gradio==3.36.1
!pip install lion-pytorch==0.0.6 lycoris_lora==1.8.0.dev6 open-clip-torch==2.20.0 prodigyopt==1.0 pytorch-lightning==1.9.0 safetensors==0.3.1 timm==0.6.12
!pip install tk==0.1.0 transformers==4.30.2 --no-deps voluptuous==0.13.1 wandb==0.15.0 xformers==0.0.20 omegaconf
!pip install torchvision==0.15.2 huggingface_hub==0.16.4 accelerate==0.21.0 numpy==1.24.4 timm==0.6.13 --force-reinstall
!pip uninstall -y jax jaxlib flax optax orbax


%cd /content
#!git clone -b 0.41.0 https://github.com/TimDettmers/bitsandbytes
#%cd /content/bitsandbytes
#!CUDA_VERSION=118 make cuda11x
#!python setup.py install

!pip install bitsandbytes==0.41.1 --prefer-binary --no-cache-dir


%cd /content
!git clone -b v1.0 https://github.com/camenduru/kohya_ss
%cd /content/kohya_ss

!python kohya_gui.py --share --headless

/content
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Using cached huggingface_hub-0.33.4-py3-none-any.whl.metadata (14 kB)
Using cached huggingface_hub-0.33.4-py3-none-any.whl (515 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.16.4
    Uninstalling huggingface-hub-0.16.4:
      Successfully uninstalled huggingface-hub-0.16.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.30.2 which is incompatible.
  Using cached torchvision-0.15.2-cp311-cp311-manylinux1_x86_64.whl.metadata (11 kB)
  Using cached huggingface_hub-0.16.4-py3-none-any.whl.metadata (12 kB)
  Using cached accelerate-0.21.0-py3-none-any.whl.metadata (17 kB)
  Using cached n

/content
/content
fatal: destination path 'kohya_ss' already exists and is not an empty directory.
/content/kohya_ss
/usr/local/lib/python3.11/dist-packages/gradio/routes.py:26: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/usr/local/lib/python3.11/dist-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/usr/local/lib/python3.11/dist-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {ex

In [ ]:
# STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [2]:
!mkdir -p /content/stable-diffusion-webui/models/Lora
!cp /content/drive/MyDrive/Angelica/Logs/last.safetensors /content/stable-diffusion-webui/models/Lora/Angelica-LoRA.safetensors


cp: cannot create regular file '/content/stable-diffusion-webui/models/Lora/Angelica-LoRA.safetensors': No such file or directory


In [3]:
# Clone AUTOMATIC1111 repo
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /content/stable-diffusion-webui

# Go into the repo
%cd /content/stable-diffusion-webui

# Skip downloading the base model (we only need LoRA testing)
!sed -i -e 's@\"sd_model_checkpoint\":.*@\"sd_model_checkpoint\": \"\",@' /content/stable-diffusion-webui/config.json

# Launch the web UI with no model (we'll load LoRA manually)
!COMMANDLINE_ARGS="--share --disable-safe-unpickle --no-half-vae" REQS_FILE="requirements_versions.txt" python launch.py


Cloning into '/content/stable-diffusion-webui'...
remote: Enumerating objects: 34968, done.
remote: Total 34968 (delta 0), reused 0 (delta 0), pack-reused 34968 (from 1)
Receiving objects: 100% (34968/34968), 35.58 MiB | 15.35 MiB/s, done.
Resolving deltas: 100% (24421/24421), done.
/content/stable-diffusion-webui
sed: can't read /content/stable-diffusion-webui/config.json: No such file or directory
Python 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
Version: v1.10.1
Commit hash: 82a973c04367123ae98bd9abdf80d9eda9b910e2
Installing clip
Cloning assets into /content/stable-diffusion-webui/repositories/stable-diffusion-webui-assets...
Cloning into '/content/stable-diffusion-webui/repositories/stable-diffusion-webui-assets'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 0), reused 20 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 132.70 KiB | 33.17 MiB

In [8]:
# Downgrade timm to a version that works with AUTOMATIC1111 and open_clip
!pip install timm==0.6.13 --force-reinstall


  Using cached PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached huggingface_hub-0.33.4-py3-none-any.whl.metadata (14 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached numpy-2.3.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pillow-11.3.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.wh

In [1]:
# Re-enter the AUTOMATIC1111 directory just to be safe
%cd /content/stable-diffusion-webui

# Relaunch with the same arguments
!COMMANDLINE_ARGS="--share --disable-safe-unpickle --no-half-vae" REQS_FILE="requirements_versions.txt" python launch.py


/content/stable-diffusion-webui
Python 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
Version: v1.10.1
Commit hash: 82a973c04367123ae98bd9abdf80d9eda9b910e2
Installing requirements
Launching Web UI with arguments: --share --disable-safe-unpickle --no-half-vae
False

===================================BUG REPORT===================================
/usr/local/lib/python3.11/dist-packages/bitsandbytes/cuda_setup/main.py:166: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/cuda_setup/main.py:166: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
The following directories listed in your path were found to be

In [1]:
# STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# 🧼 CLEAN EVERYTHING
!rm -rf /content/stable-diffusion-webui

# 🔁 Clone a fresh AUTOMATIC1111 repo
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /content/stable-diffusion-webui
!cd /content/stable-diffusion-webui

# 📦 Install minimal dependencies needed (NO bitsandbytes, no extra junk)
!pip install torch torchvision torchaudio xformers --extra-index-url https://download.pytorch.org/whl/cu118

# 📁 Make LoRA folder and copy your trained file
!mkdir -p /content/stable-diffusion-webui/models/Lora
!cp "/content/drive/MyDrive/Angelica/Output/last.safetensors" "/content/stable-diffusion-webui/models/Lora/Angelica-LoRA.safetensors"

# 🚀 Launch with clean config
!COMMANDLINE_ARGS="--share --disable-safe-unpickle --no-half-vae" python launch.py


Cloning into '/content/stable-diffusion-webui'...
remote: Enumerating objects: 34968, done.
remote: Total 34968 (delta 0), reused 0 (delta 0), pack-reused 34968 (from 1)
Receiving objects: 100% (34968/34968), 35.54 MiB | 15.73 MiB/s, done.
Resolving deltas: 100% (24409/24409), done.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 94.2 MB/s eta 0:00:00
  Attempting uninstall: xformers
    Found existing installation: xformers 0.0.20
    Uninstalling xformers-0.0.20:
      Successfully uninstalled xformers-0.0.20
  Attempting uninstal